In [1]:
from timeseries_generation import TSGenerationConfig, generate_timeseries_batches, check_batch_integrity
import xarray
from distributed import Client

client = Client('localhost:8786')
#client.restart()
client

<Client: 'tcp://146.6.192.149:8786' processes=10 threads=10, memory=465.66 GiB>

In [2]:
%%time
config_gen = TSGenerationConfig(
    "/local1/testing_timeseries/history_files/v21.LR.BSMYLEsmbb.1970-05.001/",
    "/local1/testing_timeseries/output/",
    directory_name_swaps={
        "hist": "proc/tseries"
    },
    timestep_directory_names={
        "h0": "month_1",
        "h1": "day_1",
        "h2": "hour_6",
        "h3": "hour_6",
        "h4": "hour_3",
        "h6": "month_1"
    })
# batches = config_gen.get_output_templates()
# generate_timeseries_batches(client, batches[:9], overwrite=True)
# # check_batch_integrity(batches)
#config_gen.get_output_templates()[0]


CPU times: user 169 ms, sys: 8.94 ms, total: 178 ms
Wall time: 199 ms


In [3]:
%%time
from os.path import getsize
import numpy as np


def create_order_batches(self):
    orders = []
    for index, (template, paths) in enumerate(self.output_orders):
        num_output_files = len(xarray.open_dataset(paths[0], decode_cf=False).variables)
        estimated_total_file_size = sum([getsize(path) for path in paths])

        orders.append((index, len(paths), num_output_files, int(estimated_total_file_size / (10*1024**3))))

    order_specs = np.array(orders, dtype=[('order_index', int), ('num_inputs', int), ('num_outputs', int), ('est_total_size', int)])
    order_specs = np.sort(order_specs, order=["num_outputs", "est_total_size"])
    
    unique_output_sizes = np.unique([order[1] for order in order_specs])
    
    return order_specs

specs_matrix = create_order_batches(config_gen)

CPU times: user 2.04 s, sys: 697 ms, total: 2.73 s
Wall time: 3.46 s
